In [1]:
from flask import Flask
from flask import request
import re
import google_auth
import sys
app = Flask(__name__, static_url_path='')

import pandas as pd
import numpy as np
book={}

In [2]:
def read_playbook():
    global df, book
    
    if False and "Use File":
        book={}
        df = pd.read_csv('playbook.csv')
        print('read playbook', flush=True)
        for no,s in df.iterrows():
            if not np.isnan(s["id"]):
                id=str(int(s["id"]))
                subid=str(int(s["subid"]))
                print("{} {}".format(id,subid), flush=True)
                if not id in book: book[id] = {}
                if not "description" in book[id]:
                    book[id]["description"] = s["description"]
                    book[id]["process"] = {}
                book[id]["process"][subid] = {"key":s["key"], "value":s["value"]}
    else:
        table = google_auth.fetch_playbook()
        index = 0
        for items in table:
            # skip commented line
            if len(items)==0 or items[0]=='#' or (len(items)>2 and items[0]+items[1])=="": continue
            # confirm that it has right header line
            if index==0:
                if items[0]=='id' and items[1]=='subid' and items[2]=='description' and items[3]=='key' and items[4]=='value':
                    index += 1
                    continue
                else:
                    print("playbook format broken")
                    print(items)
                    sys.exit()
            index += 1

            #print(items)
            if items[0] != "": id=items[0]
            if not id.isnumeric():
                print("id is not numeric: id={}".format(id))
                print("playbook format broken")
                sys.exit()
            subid=items[1]
            if not subid.isnumeric():
                print("subid is not numeric: id={}".format(id))
                print("playbook format broken")
                sys.exit()               

            #print("{} {}".format(id,subid), flush=True)
            if not id in book: book[id] = {}
            if not "description" in book[id]:
                book[id]["description"] = items[2] if len(items)>3 else ""
                book[id]["process"] = {}
            if len(items)>4: book[id]["process"][subid] = {"key":items[3], "value":items[4]}
            elif len(items)>3:  book[id]["process"][subid] = {"key":items[3]}

In [3]:
def mission2level(id, s):
    #some mapping algoritym
    return "2"

def account2mission(s):
    #system knows current mission in progress
    return "이미지 cropping"

In [ ]:
def render(id, s):
    if s!=s: return s
    if "ACCOUNT" in s: return "이정인"
    elif "LEVEL" in s: return mission2level(id, s)
    elif "MISSION" in s: return account2mission(s)

    if "{INPUT:user_typing}" in s: s = s.replace("{INPUT:user_typing}","입력해주세요: <input type=text name=i1>")
    if "{SELECTION:INTEGER}" in s: s = s.replace("{SELECTION:INTEGER}", "숫자를 입력해 주세요 <input type=text name=i2>")
    elif "{SELECTION:make_selection_list_from_context}" in s: s = s.replace("{SELECTION:make_selection_list_from_context}", "System wil make a list: ")
    elif "SELECTION" in s: 
        extract = re.findall(r'\{SELECTION:"(.*)"\}', s)
        print("EXTRACT", extract)
        if len(extract)>0:
            s1 = ""
            for select in extract[0].split(','):
                s1 += "<input type=radio name='r1' value=%s>%s</option>"%(select, select)
        s = s1

    return s


def show(id):
    global book
    read_playbook()
    if id == "": 
        with open("playbook.html", encoding='utf-8') as f: s = f.read()
        return s

    s = '<H3>process {} {}</H3>'.format(id, book[id]["description"])
    for proc in book[id]["process"]:
        #print(id, book[id]['process'], proc, flush=True)
        b = book[id]["process"][proc]
        s += '<p> {} {}: {}'.format(proc, b["key"], render(id, b["value"]))
        print(b, flush=True)
    s += "<p><input type=submit value='제출하기'>"
    print(s, flush=True)
    return(s)

read_playbook()
print(book, flush=True)

@app.route('/')
def hello1():
    return "hello"

@app.route('/show')
def showit():
    id = request.args.get('id', "")
    print('id=', id, flush=True )
    return show(id)


if __name__ == '__main__':
    app.run(host='0.0.0.0', port=80)

{'10': {'description': '미션종료에따라 스스로 평가 하고 peer-review 요청', 'process': {'1': {'key': 'Subject', 'value': '미션 포스트모텀 셀프평가 작성--'}, '2': {'key': 'From', 'value': '{ACCOUNT:extract_from_system}'}, '3': {'key': '레벨', 'value': '{LEVEL:make_selection_list_from_context}'}, '4': {'key': '미션', 'value': '{MISSION:make_selection_list_from_context}'}, '5': {'key': '레벨과 레벨내 해당 미션정의', 'value': '{INPUT:user_typing}{INPUT:user_typing}'}, '6': {'key': '본인이 느꼈던 난이도', 'value': '{SELECTION:"5.많이어렵다,4.조금어려운편,3.적당,2.쉬운편,1.아주쉽다"}'}, '7': {'key': '기준시간에 대항 평가', 'value': '{SELECTION:"3.시간이모자람,2.적절,1.시간이 남음"}'}, '8': {'key': '본인기준 본 미션의 적절한 기준 수행일수는?', 'value': '{SELECTION:INTEGER}'}, '9': {'key': '미션 문제해결을 위한 기술전략과 성과, 고유한 방법을 창안했다면 어떤 내용인지 설명', 'value': '{INPUT:user_typing}'}, '10': {'key': '산출물코드', 'value': '{INPUT:user_typing}'}, '11': {'key': '사용한 기술요소', 'value': '{{SELECTION:make_selection_list_from_context},{INPUT:user_typing}}'}}}, '11': {'description': '동료의 미션종료 peer-review 참여요청 수락', 'process': {'1': {'ke

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:80/ (Press CTRL+C to quit)


id= 


127.0.0.1 - - [12/Apr/2021 00:13:23] "GET /show HTTP/1.1" 200 -
127.0.0.1 - - [12/Apr/2021 00:13:23] "GET /level_mission.jpg HTTP/1.1" 404 -


id= 


127.0.0.1 - - [12/Apr/2021 00:49:35] "GET /show HTTP/1.1" 200 -
127.0.0.1 - - [12/Apr/2021 00:49:35] "GET /level_mission.jpg HTTP/1.1" 404 -


In [23]:
a=[1,2,3,4,5]
k=iter(a)

In [24]:
k

In [25]:
next(k)

1

In [30]:
next(k)

StopIteration: 